In [3]:
!wget https://www.kaggle.com/kemical/kickstarter-projects/download

/bin/bash: kaggle: command not found


0

In Linux installing Java 8 as the following will help:

sudo apt install openjdk-8-jdk
Then set the default Java to version 8 using:

sudo update-alternatives --config java
***************** : 2 (Enter 2, when it asks you to choose) + Press Enter

In [1]:
!rm -f ks-projects-201612.csv kickstarter-projects.zip ks-projects-201801.csv

In [3]:
# https://github.com/Kaggle/kaggle-api#datasets
# https://sparkbyexamples.com/spark/spark-installation-on-linux-ubuntu/

!pip install --user kaggle
!pip install --user pyspark

In [3]:
!kaggle datasets download -d kemical/kickstarter-projects

100%|██████████████████████████████████████| 36.8M/36.8M [00:03<00:00, 11.1MB/s]
100%|██████████████████████████████████████| 36.8M/36.8M [00:03<00:00, 10.6MB/s]


In [4]:
!ls

draft.ipynb  kickstarter-projects.zip  README.md


In [5]:
!unzip kickstarter-projects.zip

Archive:  kickstarter-projects.zip
  inflating: ks-projects-201612.csv  
  inflating: ks-projects-201801.csv  


In [6]:
!ls

draft.ipynb		  ks-projects-201612.csv  README.md
kickstarter-projects.zip  ks-projects-201801.csv


In [7]:
!rm -f ks-projects-201612.csv kickstarter-projects.zip #ks-projects-201801.csv

In [8]:
!ls

draft.ipynb  ks-projects-201801.csv  README.md


In [2]:
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [4]:
campaigns = (
    spark.read.format("csv").options(header=True).load("ks-projects-201801.csv")
)

In [5]:
 campaigns = campaigns.select("ID", "name", "category", "deadline", "launched",
                              "country","usd_pledged_real","state","usd_goal_real")

In [6]:
campaigns.show(n=5)

+----------+--------------------+--------------+----------+-------------------+-------+----------------+--------+-------------+
|        ID|                name|      category|  deadline|           launched|country|usd_pledged_real|   state|usd_goal_real|
+----------+--------------------+--------------+----------+-------------------+-------+----------------+--------+-------------+
|1000002330|The Songs of Adel...|        Poetry|2015-10-09|2015-08-11 12:12:28|     GB|            0.00|  failed|      1533.95|
|1000003930|Greeting From Ear...|Narrative Film|2017-11-01|2017-09-02 04:43:57|     US|         2421.00|  failed|     30000.00|
|1000004038|      Where is Hank?|Narrative Film|2013-02-26|2013-01-12 00:20:50|     US|          220.00|  failed|     45000.00|
|1000007540|ToshiCapital Reko...|         Music|2012-04-16|2012-03-17 03:24:11|     US|            1.00|  failed|      5000.00|
|1000011046|Community Film Pr...|  Film & Video|2015-08-29|2015-07-04 08:35:03|     US|         1283.00|

In [36]:
# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="name", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="term_frequency")
idf = IDF(minDocFreq=3, inputCol=hashingTF.getOutputCol(), outputCol="embedding")

dataPreparationPipeline = Pipeline(stages=[tokenizer, hashingTF])

# Fit the pipeline to training documents.
prepared_campaigns = dataPreparationPipeline.fit(campaigns).transform(campaigns)

In [37]:
from pyspark.sql.functions import isnan, when, count, col
campaigns.select([count(when(isnan(c), c)).alias(c) for c in campaigns.columns]).show()

+---+----+--------+--------+--------+-------+----------------+-----+-------------+
| ID|name|category|deadline|launched|country|usd_pledged_real|state|usd_goal_real|
+---+----+--------+--------+--------+-------+----------------+-----+-------------+
|  0|   0|       0|       0|       0|      0|               0|    0|            0|
+---+----+--------+--------+--------+-------+----------------+-----+-------------+



In [38]:
prepared_campaigns.select("name", "words", "term_frequency").show(n=5)

+--------------------+--------------------+--------------------+
|                name|               words|      term_frequency|
+--------------------+--------------------+--------------------+
|The Songs of Adel...|[the, songs, of, ...|(262144,[6946,511...|
|Greeting From Ear...|[greeting, from, ...|(262144,[73198,97...|
|      Where is Hank?|  [where, is, hank?]|(262144,[59050,75...|
|ToshiCapital Reko...|[toshicapital, re...|(262144,[27576,10...|
|Community Film Pr...|[community, film,...|(262144,[1168,172...|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [39]:
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="embedding")
dataPreparationPipeline = Pipeline(stages=[idf])
prepared_campaigns = dataPreparationPipeline.fit(prepared_campaigns).transform(prepared_campaigns)
prepared_campaigns.select("name", "words", "term_frequency","embedding").show(n=5)

Py4JJavaError: An error occurred while calling o497.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 13, desktop-lkaf740-2.home, executor driver): org.apache.spark.SparkException: Failed to execute user defined function(Tokenizer$$Lambda$2318/452863628: (string) => array<string>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:729)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.It

In [22]:
from pyspark.ml.feature import Word2Vec

documentDF = campaigns.select("name")


In [25]:
type(documentDF)
documentDF.show(5)

+--------------------+
|                name|
+--------------------+
|The Songs of Adel...|
|Greeting From Ear...|
|      Where is Hank?|
|ToshiCapital Reko...|
|Community Film Pr...|
+--------------------+
only showing top 5 rows



In [24]:
# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="name", outputCol="result")
model = word2Vec.fit(documentDF)
result = model.transform(documentDF)

for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

IllegalArgumentException: requirement failed: Column name must be of type equal to one of the following types: [array<string>, array<string>] but was actually of type string.

In [32]:
from pyspark.ml.linalg import DenseVector
df = spark.createDataFrame([(DenseVector([1.0, 2.0]),),(DenseVector([0.0, 1.0]),), (DenseVector([3.0, 0.2]),)], ["tf"])
idf = IDF(minDocFreq=3)
idf.setInputCol("tf")

IDF_6c68e4bb6b55

In [33]:
idf.setOutputCol("idf")

IDF_6c68e4bb6b55

In [34]:
model = idf.fit(df)
model.setOutputCol("idf")
model.getMinDocFreq()
model.idf

DenseVector([0.0, 0.0])

In [35]:
model.docFreq

[0, 3]

In [40]:
model.numDocs == df.count()
model.transform(df).head().idf

DenseVector([0.0, 0.0])

In [ ]:
DenseVector([0.0, 0.0])
idf.setParams(outputCol="freqs").fit(df).transform(df).collect()[1].freqs
DenseVector([0.0, 0.0])
params = {idf.minDocFreq: 1, idf.outputCol: "vector"}
idf.fit(df, params).transform(df).head().vector
DenseVector([0.2877, 0.0])
idfPath = temp_path + "/idf"
idf.save(idfPath)
loadedIdf = IDF.load(idfPath)
loadedIdf.getMinDocFreq() == idf.getMinDocFreq()
True
modelPath = temp_path + "/idf-model"
model.save(modelPath)
loadedModel = IDFModel.load(modelPath)
loadedModel.transform(df).head().idf == model.transform(df).head().idf